<figure>
  <IMG SRC="https://gitlab.tudelft.nl/mude/public/-/raw/main/tu-logo/TU_P1_full-color.png" WIDTH=150 ALIGN="right">
</figure>
<figure>
  <IMG SRC="https://gitlab.tudelft.nl/mude/public/-/raw/main/mude-logo/MUDE_Logo-small.png" WIDTH=150 ALIGN="right">
</figure>

### CEGM1000 Modelling, Uncertainty and Data for Engineers
    
#### Sensing and observation theory : Overall Model test

In the following exercise, we will look at the curve fitting problem to a synthetic sea level rise dataset. The dataset contains yearly sea level measurements over 20 years (in total 20 observations). The time of observations are given in years as [1, 2, 3, ..., 20]. The observations are assumed to be normally distributed and have a precision of $\sigma=2$ cm. The sea level at time zero is unknown and should also be estimated together with the other parameters. 

**In this exercise, we will:**
1. Import the data,  create the design matrix $\mathrm{A}$, and the covariance matrix $\Sigma_Y$ (assuming the linear trend model)
2. Apply BLUE and overall model test

**We will look at 3 cases where the null-hypothesis is rejected:**
1. Dataset is corrupted by one large outlier.
2. We apply an incorrect functional model.
3. We use a too optimistic value for the precision.  

If we detect that the overall model test (i.e., our null hypothesis) is rejected, we would in practice need to apply the Generalized Likelihood Ratio test to test an alternative hypothesis against the null hypothesis, to decide whether it is sufficiently more likely.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.stats.distributions import chi2

%matplotlib inline
plt.rcParams.update({'font.size': 14})

Because we will be using the BLUE multiple times throughout this notebook, it is convenient to create a function for it:

In [2]:
def blue(A, y, Sigma_Y):
    """Calculate the estimates of the unknown parameters and the corresponding
    covariance matrix using the BLUE.
    
    Parameters
    ----------
    A: design matrix.
    y: observations.
    Sigma_Y: covariance matrix of the observables.
    
    Returns
    -------
    xhat: estimates of the unknown parameters.
    Sigma_Xhat: covariance matrix of the estimates.
    yhat: adjusted observations.
    Sigma_Yhat: covariance matrix of adjusted observations.
    e_hat: residuals.
    Sigma_Ehat: covariance matrix of residuals.
    """

    Sigma_Xhat = np.linalg.inv(A.T @ np.linalg.inv(Sigma_Y) @ A)
    xhat = Sigma_Xhat @ A.T @ np.linalg.inv(Sigma_Y) @ y
    
    yhat = A @ xhat  
    Sigma_Yhat = A @ Sigma_Xhat @ A.T
    
    e_hat = y - yhat
    Sigma_Ehat = Sigma_Y - Sigma_Yhat
    
    return xhat, Sigma_Xhat, yhat, Sigma_Yhat, e_hat, Sigma_Ehat

Make a function to calculate the width of the confidence interval:

In [3]:
def confidence_interval(conf_level, Sigma):
    """Calculate lower and upper bounds of the confidence interval.
    
    Parameters
    ----------
    conf_level: confidence level (=1-alpha), e.g., 0.95.
    Sigma: covariance matrix of the variable for which to calculate the
             confidence interval.

    Returns
    -------
    cint: half of the width of the confidence interval.

    Examples
    --------
    The lower and upper bounds of the confidence interval can be calculated as:
    y_lower = y - cint
    y_upper = y + cint
    """
    
    alpha = 1 - conf_level
    k = norm.ppf(1 - alpha/2)
    cint =  k * (np.sqrt(np.diagonal(Sigma)))

    return cint

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p><b>
Complete the function <code>overall_model_test</code>.
</b> </p>    
</div>

In [5]:
def overall_model_test(alpha, ehat, Sigma_Y, q):
    """Performs the overall model test and prints the outcome.
    
    Parameters
    ----------
    alpha: false alarm probability
    ehat: residuals.
    Sigma_Y: covariance matrix of the observables.
    q: degrees of freedom of Chi2-distribution.
    """

    k = ?
    Tq = ?
    
    if Tq < k:
        print(f"(T = {Tq:.1f}) < (K = {k:.1f}), OMT is accepted.")
    else:
        print(f"(T = {Tq:.1f}) > (K = {k:.1f}), OMT is rejected.")

Make a function that plots all the results:

In [4]:
def plot_results(t, A, y, Sigma_Y, conf_level, alpha):#(t, y, yhat, yhat_int, ehat, ehat_int, conf_level, alpha):
    """Produce plots of the (adjusted) observations and the residuals,
    including the confidence bounds.
    
    Parameters
    ----------
    t: times.
    y: observations.
    A: design matrix.
    Sigma_Y: covariance matrix of the observables.
    conf_level: confidence level.
    """
    
    xhat, Sigma_Xhat, yhat, Sigma_Yhat, ehat, Sigma_Ehat = blue(A, y, Sigma_Y)
    
    yhat_int = confidence_interval(conf_level, Sigma_Yhat)
    ehat_int = confidence_interval(conf_level, Sigma_Ehat)
    
    overall_model_test(alpha, ehat, Sigma_Y, len(y)-len(xhat))

    plt.figure(figsize=(12,6))
    
    plt.subplot(121)
    plt.plot(t, y,'r.',markersize=15, label='observations')
    plt.plot(t, yhat, linewidth=3, label='fitted model')
    plt.plot(t, yhat-yhat_int, 'r--', linewidth=2,
             label='{} % conf.'.format(conf_level*100))
    plt.plot(t, yhat+yhat_int, 'r--', linewidth=2)
    plt.grid()
    plt.xlabel('time [years]')
    plt.ylabel('Sea level [cm]')
    plt.legend(loc='best')
    
    plt.subplot(122)
    plt.plot(t, ehat, 'r.',markersize=15, label='residuals')
    plt.plot(t, ehat_int, 'r--', linewidth=2,
             label='{} % conf.'.format(conf_level*100))
    plt.plot(t, - ehat_int, 'r--', linewidth=2)
    plt.grid()
    plt.xlabel('time [years]')
    plt.ylabel('Residuals [cm]')
    plt.legend(loc='best')

### Case 0: linear trend model is correct

First, import the data and compute the number of observations ```m```, vector with observation times ```t```, standard deviation of the measurements ```std_y```, covariance matrix of observations ```Sigma_Y```, design matrix ```A```.

In [6]:
y_0 = np.genfromtxt('Dataset_0.txt')
m = len(y_0)
t = np.arange(1,m+1)

std_y = 2 # [cm]
Sigma_Y = std_y**2 * np.eye(m)
A = np.column_stack((np.ones(m), t))

Perform BLUE and the overall model test with a false alarm probability of 5% and plot results of BLUE:

In [7]:
plot_results(t, A, y_0, Sigma_Y, conf_level = 0.95, alpha = 0.05)

NameError: name 'red' is not defined

### Case 1: linear trend model, outlier in data

In Dataset_1 one of the observations is an outlier... We will apply BLUE and perform the overall model test with this new dataset.

In [ ]:
y_1 = np.genfromtxt('Dataset_1.txt')
plot_results(t, A, y, Sigma_Y, conf_level = 0.95, alpha = 0.05)

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p><b>
Q1: Why are more residuals negative in this case?
</b> </p>    
</div>

### CASE 2: linear trend model is too simplistic

For Dataset_2 the linear trend model assumptions is not correct... We will apply BLUE and perform the overall model test with this new dataset.

In [ ]:
y_2 = np.genfromtxt('Dataset_2.txt')
plot_results(t, A, y_2, Sigma_Y, conf_level=0.95, alpha = 0.05)

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p><b>

Q2: How can you see from the residuals plot that the linear model is too simplistic?

Q3: What would be a better model? Try yourself by completing the code below.
</b> </p>    
</div>

In [ ]:
A_2 = ?
plot_results(t, A_2, y_2, Sigma_Y, conf_level=0.95, alpha = 0.05)

### CASE 3: linear trend model, too optimistic about precision

The observations in Dataset_3 are less precise than before... We will apply BLUE and perform the overall model test with this new dataset.

In [ ]:
y_3 = np.genfromtxt('Dataset_3.txt')
plot_results(t, A, y_3, Sigma_Y, conf_level=0.95, alpha = 0.05)

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p><b>

Q4: How can you see that we were too optimistic about the precision of the observations? Play with the factor below to see if indeed the model is accepted if we choose a larger value for the precision.
</b> </p>    
</div>

In [ ]:
factor = ?
plot_results(t, A, y_3, factor * Sigma_Y, conf_level=0.95, alpha = 0.05)